In [4]:
import requests as req
import pandas as pd
import requests
import time
import re
from bs4 import BeautifulSoup as bsp
from selenium import webdriver
from seleniumwire import webdriver  # blinker == 1.7.0
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.edge.service import Service as EdgeService
from selenium.webdriver.edge.options import Options as EdgeOptions
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from fake_useragent import UserAgent
from googlesearch import search
from datetime import datetime


base = 'https://www.amazon.com'
thirdWheel = '/s?k='
date_pattern = r"\b(January|February|March|April|May|June|July|August|September|October|November|December) \d{1,2}, \d{4}\b"
number_pattern = r"\d+"

In [ ]:
# session = requests.Session()
# session.headers.update({"User-Agent": UserAgent().random})
driver = webdriver.Chrome()
# driver = webdriver.Firefox(service = service, options = options)
driver.get(base)
#   It will scroll to right above the footer of the page then scoll to the top then back to the bottom untill there is no new items being loaded
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(10) 
html = driver.page_source
driver.quit()          
soup = bsp(html, 'lxml')
boxes = soup.find_all('a', class_ = 'a-link-normal _fluid-quad-image-label-v2_style_centerImage__30wh- aok-block image-window')
print(boxes)
links = []
urls = []
categ = []
for box in boxes:
    actual = box.get('href')
    url = base + actual
    print("Url : ", url)
    urls.append(url)
    som = box.get('aria-label')
    link = som.replace(' ', '+')
    link = base + thirdWheel + link
    print("Links : ", link)
    links.append(link)
    categ.append(som)
    print(som)


In [6]:
def get_valid_proxies():
    proxy_list_url = 'https://free-proxy-list.net/'
    response = requests.get(proxy_list_url)
    soup = bsp(response.text, 'html.parser')
    proxy_data = []
    rows = soup.find_all('tr')[1:]
    for row in rows:
        columns = row.find_all('td')
        if len(columns) >= 8:
            ip_address = columns[0].text.strip()
            google_enabled = columns[5].text.strip().lower() == 'yes'
            https_enabled = columns[6].text.strip().lower() == 'yes'
            last_checked = columns[7].text.strip()
            if (last_checked.endswith('mins ago') and int(last_checked.split(' ')[0]) < 15) or last_checked.endswith('hours ago'):
                if google_enabled or https_enabled:
                    proxy_data.append({'ip_address': ip_address, 'google_enabled': google_enabled, 'https_enabled': https_enabled})

    return proxy_data


In [7]:
def rotate_user_agent(proxy):
    if proxy:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
            'http': f'http://{proxy}',
            'https': f'https://{proxy}'
        }
    else:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
        }
    return headers

In [ ]:
products = []
categories = []
cat = 0
me = False
# proxies = get_valid_proxies()
for url in urls:
    total_samples = 0
    myurl = url
    while myurl:
        # proxy = proxies[total_samples % len(proxies)] if proxies else None
        options = {
            'headers' : {
                "User-Agent": UserAgent().random
            }
        }
        print("VISITING : ", myurl)
        # sdsd = Options()
        # sdsd.add_argument("--headless")  # Run in background
        # sdsd.add_argument("--disable-gpu")
        # sdsd.add_argument("--no-sandbox")
        # sdsd.add_argument("--disable-dev-shm-usage")
        driver = webdriver.Chrome(seleniumwire_options = options)
        # driver = webdriver.Firefox()
        driver.get(myurl)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(10)
        html = driver.page_source
        driver.quit()
        # headers = rotate_user_agent(proxy)
        # response = req.get(myurl, headers = headers)
        # session = requests.Session()
        # session.headers.update({"User-Agent": UserAgent().random})
        # response = req.get(myurl, headers = {"User-Agent": UserAgent().random})
        # if response.status_code == 200:
            # soup = bsp(response.text, 'lxml')
        soup = bsp(html, 'lxml')
        # print(soup)
        productes = soup.find_all('a', class_ = 'a-link-normal s-line-clamp-4 s-link-style a-text-normal')
        
        if not productes:
            productes = soup.find_all('a' , class_= 'a-link-normal s-line-clamp-2 s-link-style a-text-normal')
        print(productes)
        for product in productes:
            product = base + product.get('href')
            products.append(product)
            if cat in range(len(categ)):
                categories.append(categ[cat])
            print(product)
            csv_file = 'Amazon.csv'
            info = {"Products": products, "Categories": categories}
            new_df = pd.DataFrame(info)
            if not me:
                new_df.to_csv(csv_file, mode='a', index = False)
                me = True
            else:
                new_df.to_csv(csv_file, mode='a', header=False, index = False)
            print("|||||||||||||||||||||||||||| Data appended successfully with continued index. |||||||||||||||||||||||||||")
        
        total_samples += 1
        nextP = soup.find('a', class_ = 's-pagination-item s-pagination-next s-pagination-button s-pagination-button-accessibility s-pagination-separator')
        if nextP:
            myurl = base + nextP.get('href')
        if total_samples >= 3:
            break
    cat += 1



In [ ]:
agent = {
    "Main_Category": [],
    "Title": [],
    "AvgRating": [],
    "RatingNo": [],
    "Features": [[]],
    "Description": [[]],
    "Price": [],
    "Images": [[]],
    "Videos": [[]],
    "Store": [],
    "Categories": [],
    "Details": [[]],
    "ParentAsin": [],
    "BoughtTogether": [[]]
}

comment = {
    
}

csv_file = 'G:\\Amazon.csv'
df = pd.readcsv(csv_file)
samples = 0
for row in df.iterrows():
    options = {
            'headers' : {
                "User-Agent": UserAgent().random
            }
        }
    link = row['Products']
    print("VISITING : ", link)
    driver = webdriver.Chrome(seleniumwire_options = options)
    driver.get(link)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    driver.execute_script("window.scrollTo(0, 0);")
    time.sleep(10)
    html = driver.page_source
    driver.quit()
    soup = bsp(html, 'lxml')

    feat = []
    img = []
    vid = []
    det = []

    # extracting Category
    category = row['Categories']

    # extracting Title
    try:
        titleBoard = soup.find('span', id = 'productTitle')
        title = titleBoard.text.strip()
    except:
        title = ""

    # extracting ratings
    try:
        ratingBoard = soup.find('div', id = 'averageCustomerReviews')
        # extracting average rating
        avgRatingBoard = ratingBoard.find('span', class_ = 'a-size-base a-color-base')
        avgRating = avgRatingBoard.text.strip()
        # extracting rating number
        ratingNoBoard = ratingBoard.find('span', id = 'acrCustomerReviewText')
        ratingNo = ratingNoBoard.text.strip()
        junk = " ratings"
        ratingNo = ratingNo.replace(junk, "")
    except:
        avgRating = ""
        ratingNo = ""

    # extracting price
    try:
        priceBoard = soup.find('div', class_ = 'a-section a-spacing-none aok-align-center aok-relative').find('span', class_ = 'aok-offscreen')
        price = priceBoard.text.strip()[0:5]
    except:
        price = ""

    # exracting media
    try:
        mediaBoard = soup.find('ul', class_ = 'a-unordered-list a-nostyle a-button-list a-vertical a-spacing-top-micro regularAltImageViewLayout')
        if not mediaBoard:
            mediaBoard = soup.find('ul', class_ = 'a-unordered-list a-nostyle a-button-list a-vertical a-spacing-top-micro gridAltImageViewLayoutIn1x7')
        media = mediaBoard.find_all('img')
        images = [img['src'] for img in media]
        vids = images[-1]
        images = images[0:-1]
    except:
        images = []
        vids = []
    

    # extracting the store
    try:
        storeBoard = soup.find('tr', class_ = 'a-spacing-small po-brand').find('td', class_ = 'a-span9')
        store = storeBoard.text.strip()
    except:
        store = ""

    # extracting features
    try:
        featuresBoard = soup.find('ul', class_ = 'a-unordered-list a-vertical a-spacing-mini').find_all('il')
        features = [feature.text.strip() for feature in featuresBoard]
    except:
        features = []
        
    # extracting details
    try:
        detailsBoard = soup.find('div', class_ = 'a-expander-content a-expander-section-content a-section-expander-inner').find('table', class_ = 'a-keyvalue prodDetTable').find_all('tr')
    except:
        try:
            detailsBoard = soup.find('div', id = 'detailBullets_feature_div').find_all('li')
        except:
            try:
                detailsBoard = soup.find('table', id = 'productDetails_detailBullets_sections1').find_all('tr')
            except:    
                try:
                    detailsBoard = soup.find('ul', class_ = 'a-unordered-list a-nostyle a-vertical a-spacing-none detail-bullet-list').find_all('tr')
                except:
                    detailsBoard = []
                    
    # extracting asin
    try:
        asinBoard = soup.find('tr', text=lambda x: x.strip() == 'ASIN', class_='prodDetSectionEntry')
        asin = asinBoard.text.strip()
    except:
        asin = ""
    
    # extracting description
    try:
        descriptionBoard = soup.find('ul', id = 'a-nostyle').find_all('div', class_ = 'a-fixed-left-grid-col a-col-right')
        descriptionBoardB = soup.find('table', class_ = 'a-normal a-spacing-micro')
        descriptions = descriptionBoardB.find_all('td', class_ = 'a-span9')
        description = [dec.text.strip() for dec in descriptions]
    except:
        description = ""

    # extracting bought together
    boughtTogetherBoard = soup.find('div', id = 'a-cardui _c3AtZ_new-thumbnail-box_1W9Ku _c3AtZ_two-item-thumbnail-box_7kF95')
    if not boughtTogetherBoard:
        boughtTogetherBoard = soup.find('div', class_ = 'a-cardui _p13n-desktop-sims-fbt_fbt-desktop_new-thumbnail-box__36bD3 _p13n-desktop-sims-fbt_fbt-desktop_two-item-thumbnail-box__jV2am')
    if not boughtTogetherBoard:
        boughtTogetherBoard = soup.find('div', class_ = 'a-cardui _p13n-desktop-sims-fbt_fbt-desktop_new-thumbnail-box__36bD3')
    boughtTogetherS = boughtTogetherBoard.find_all('a', class_ = 'a-link-normal a-text-normal', href = True)
    if not boughtTogetherS:
        boughtTogetherS = boughtTogetherBoard.find_all('a', class_ = 'a-link-normal', href = True)        
    boughtTogether = [base + link['href'] for link in boughtTogetherS]

    # extracting further categories
    catgoriesBoard = soup.find_all('div', id= 'tp-inline-twister-dim-values-container')
    categoriesBoard = categoriesBoard[1]


    ############################################################################################################################################################
    # extracting reviews
    # reviews from US
    reviewsUS = soup.find_all('ul', id = 'x`t').find_all('li')
    reviewsGlobal = soup.find_all('ul', id = 'cm-cr-global-review-list').find_all('li')
    reviews = reviewsUS + reviewsGlobal
    for review in reviews:
        # extracting rating
        reviewRatingBoard = review.find('i', class_ = 'review-star-rating').find('span', class_ = 'fa-icon-alt')
        reviewRating = reviewRatingBoard.text[0:3]

        # extracting title
        reviewTitleBoard = review.find('a', class_ = 'a-size-base a-link-normal review-title a-color-base review-title-content a-text-bold')
        reviewTitles = reviewTitleBoard.find_all('span')
        reviewTitle = reviewTitles[-1].text.strip()

        # extracting Text
        reviewTextBoard = review.find('div', class_ = 'a-expander-content reviewText review-text-content a-expander-partial-collapse-content')
        reviewText = reviewTextBoard.text.strip()

        # extracting images
        reviewImages = review.find('div', class_ = 'review-image-tile-section').find_all('img')
        reviewImages = [img['src'] for img in reviewImages]

        # extracting ASIN
        reviewAsin = asin

        # extracting parent ASIN
        reviewParentAsin = asin

        # eaxtracting username
        username = review.find('span', class_ = 'a-profile-name').text.strip()

        # extracting date
        reviewTimeBoard = review.find('span', class_ = 'a-size-base a-color-secondary review-date')
        reviewTimes = reviewTimeBoard.text
        reviewTimes = re.search(date_pattern, reviewTimes)
        reviewTiming = datetime.strptime(reviewTimes, "%B %d, %Y")
        reviewTime = int(reviewTiming.timestamp())

        verified = False
        if review.find('span', class_ = 'a-size-mini a-color-state a-text-bold'):
            verified = True


        reviewHelpful = review.find('span', class_ = 'a-size-base a-color-tertiary cr-vote-text')
        reviewHelpful = reviewHelpful.text
        haha = "One"
        helpful = 0
        Helpfulness = re.search(number_pattern, reviewHelpful)
        if Helpfulness:
            helpful = int(Helpfulness.group())
        elif haha in reviewHelpful:
            helpful = 1


    # reviews from all over the globe

    





In [ ]:


# Define the date strings
date_str1 = "June 22, 2016"
date_str2 = "September 30, 2023"

# Convert the date strings to datetime objects
date1 = datetime.strptime(date_str1, "%B %d, %Y")
date2 = datetime.strptime(date_str2, "%B %d, %Y")

# Compare the dates
if date1 > date2:
    print(f"{date_str1} is greater than {date_str2}")
else:
    print(f"{date_str1} is not greater than {date_str2}")

In [ ]:
# Date string in the format "Month Day, Year"
date_str = "January 17, 2025"

# Parse the string into a datetime object
date_obj = datetime.strptime(date_str, "%B %d, %Y")

# Convert to Unix time
unix_time = int(date_obj.timestamp())

print(f"Unix Time: {unix_time}")



In [ ]:
# Example statement
statement = "The item costs 700 dollars, and the discount is."

# Regular expression to match one or more digits
number_pattern = r"\d+"

# Find the first number in the statement
match = re.search(number_pattern, statement)

if match:
    number = int(match.group())  # Convert the matched number to an integer
    print(f"Extracted number: {number}")
else:
    print("No number found.")
